# Numerical optimization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy import optimize

## Example 1

Consider the minimization of the function

$$ f(x) = \exp[ (x-0.7)^2 ] $$

Analytically, we know the minimimum of $f$ is at $x=0.7$. To verify, we can plot $f(x)$ to see:

In [ ]:
def f(x):
    return -np.exp(-(x - 0.7)**2)

In [ ]:
x=np.linspace(0.4, 1.0, 101)
plt.plot(x, f(x), 'r-')
plt.plot([0.7],f(0.7), 'ko')

In [ ]:
# Use minimizer to get the minimum point

result = optimize.minimize_scalar(f)

In [ ]:
# always check the documentation
#optimize.minimize_scalar?

In [ ]:
result.success

In [ ]:
result.x

## Example 2

Consider another function

$$ f(x) = (x-x_0)^2 + \varepsilon \exp[ -5*(x-0.5-x_o)^2 ] $$

where $\varepsilon$ is 0 or 1. When $\varepsilon=0$, $f(x)$ is convex. When $\varepsilon=1$, $f(x)$ is non-convex.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

x = np.linspace(-1, 3, 100)
x_0 = np.exp(-1)

def f(x):
    return (x - x_0)**2 + epsilon*np.exp(-5*(x - .5 - x_0)**2)

In [ ]:
for epsilon in (0,1):
    plt.figure(figsize=(3, 2.5))
    plt.axes([0, 0, 1, 1])
    plt.plot(x, f(x),'k-')
    plt.text(0, 5, r'$\varepsilon=$' + '{}'.format(epsilon),size=16)
    plt.ylabel(r'$f(x)$')
    plt.xlabel(r'$x$')

We will use the following functions from numpy:
- finfo(): https://numpy.org/doc/stable/reference/generated/numpy.finfo.html

For example:

In [ ]:
# For 64-bit binary floats in the IEEE-754 standard, eps = 2**-52, approx. 2e-16, i.e.
np.finfo(1.).eps

In [ ]:
for epsilon in (0, 1):
    plt.figure(figsize=(8, 3))
    plt.axes([0, 0, 1, 1])

    # A convex function
    plt.subplot(1,2,1)
    plt.text(0, 5, r'$\varepsilon=$' + '{}'.format(epsilon),size=16)
    plt.plot(x, f(x), color='lightgray', linewidth=2)
    

    # Apply brent method. To have access to the iteration, do this in an
    # artificial way: allow the algorithm to iter only once
    all_x = list()
    all_y = list()
    for iter in range(30):
        result = optimize.minimize_scalar(f, bracket=(-5, 2.9, 4.5), method="Brent",
                    options={"maxiter": iter}, tol=np.finfo(1.).eps)
        if result.success:
            print('Converged at ', iter, ', x=', result.x)
            break

        this_x = result.x
        all_x.append(this_x)
        all_y.append(f(this_x))
        if iter < 6:
            # print iteration number near [this_x,f(this_x)]
            plt.text(this_x - .05*np.sign(this_x) - .05,
                    f(this_x) + 1.2*(.3 - iter % 2), iter + 1,
                    size=12)

    plt.plot(all_x[:10], all_y[:10], 'k+', markersize=12, markeredgewidth=2)

    plt.plot(all_x[-1], all_y[-1], 'rx', markersize=12)
    plt.axis('off')
    plt.ylim(ymin=-1, ymax=8)

    #plt.figure(figsize=(4, 3))
    plt.subplot(1,2,2)
    # error = (current value) - (final value)
    plt.semilogy(np.abs(all_y - all_y[-1]), '--ko', linewidth=2)
    plt.ylabel('Error on f(x)')
    plt.xlabel('Iteration')
    plt.tight_layout()

## Nonlinear least squares

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit

Consider the function

In [ ]:
def f(t, omega, phi):
    return np.cos(omega * t + phi)

In [ ]:
N = 101
x = np.linspace(0, 3, N)
omega = 1.5
phi = 1

In [ ]:
y = f(x, omega, phi) + .1*np.random.normal(size=len(x))
plt.figure()
plt.plot(x,y,'rx')

In [ ]:
res = optimize.curve_fit(f, x, y)   

In [ ]:
omega, phi = res[0]

In [ ]:
plt.plot(x,f(x,omega,phi),'k-')
plt.plot(x,y,'r+')

Consider a different omega value

### omega = 2

In [ ]:
omega = 2
phi = 1
y = f(x, omega, phi) + .1*np.random.normal(size=len(x))
plt.figure()
plt.plot(x,y,'rx')

In [ ]:
res = optimize.curve_fit(f, x, y)   
omega, phi = res[0]
plt.figure()
plt.plot(x,f(x,omega,phi),'k-')
plt.plot(x,y,'r+')

In [ ]:
res

### omega = 3

In [ ]:
omega = 3
phi = 1
y = f(x, omega, phi) + .1*np.random.normal(size=len(x))
res = optimize.curve_fit(f, x, y)   
omega, phi = res[0]
plt.figure()
plt.plot(x,f(x,omega,phi),'k-')
plt.plot(x,y,'r+')

In [ ]:
res

In [ ]:
# If we specify the right bounds...

res = optimize.curve_fit(f, x, y, bounds = ([0,1],[2,2]))   
#res = optimize.curve_fit(f, x, y, bounds = ([2,1],[4,2]))   
omega, phi = res[0]
plt.figure()
plt.plot(x,f(x,omega,phi),'k-')
plt.plot(x,y,'r+')

In [ ]:
res

## Constrained optimization

### Example 1

Consider the two-dimensional function

$$ f(x_0, x_1) = \sqrt{ (x_0-3)^2 + (x_1 - 2)^2 } $$

subjecto to the constrains of

$$ -1.5 \leq x_0 \leq 1.5$$
$$ -1.5 \leq x_1 \leq 1.5$$

**Note 1**  Without the constraints the $f(x_0,x_1)$ would occur at $x_0=3$ and $x_1=2$.

**Note 2** The derivatives (or gradient) of $f$ with respect to $x_0$ and $x_1$ are (Exercise: verify this!)

$$ \frac{\partial f}{\partial x_0} = \frac{x_0 - 3}{f}$$
$$ \frac{\partial f}{\partial x_1} = \frac{x_1 - 2}{f}$$

See the function `f_prime(x)` below.

In [ ]:
def f(x):
   return np.sqrt((x[0] - 3)**2 + (x[1] - 2)**2)

In [ ]:
res = optimize.minimize(f, np.array([0, 0]), bounds=((-1.5, 1.5), (-1.5, 1.5))) 
res

In [ ]:
x, y = np.mgrid[-2.9:5.8:.05, -2.5:5:.05]
x = x.T
y = y.T

for i in (1, 2):
    # Create 2 figure: only the second one will have the optimization
    # path
    plt.figure(i, figsize=(3, 2.5))
    plt.clf()                           # clear the current figure
    plt.axes([0, 0, 1, 1])

    contours = plt.contour(np.sqrt((x - 3)**2 + (y - 2)**2),
                        extent=[-3, 6, -2.5, 5],
                        cmap=plt.cm.gnuplot)
    plt.clabel(contours,
            inline=1,
            fmt='%1.1f',
            fontsize=14)
    plt.plot([-1.5, -1.5,  1.5,  1.5, -1.5],
            [-1.5,  1.5,  1.5, -1.5, -1.5], 'k', linewidth=2)
    plt.fill_between([ -1.5,  1.5],
                    [ -1.5, -1.5],
                    [  1.5,  1.5],
                    color='.8')
    plt.axvline(0, color='k')
    plt.axhline(0, color='k')

    plt.text(-.9, 4.4, '$x_2$', size=20)
    plt.text(5.6, -.6, '$x_1$', size=20)
    plt.axis('equal')
    plt.axis('off')

# And now plot the optimization path
accumulator = list()

def f(x):
    # Store the list of function calls
    accumulator.append(x)
    return np.sqrt((x[0] - 3)**2 + (x[1] - 2)**2)


# We don't use the gradient, as with the gradient, L-BFGS is too fast,
# and finds the optimum without showing us a pretty path
def f_prime(x):
    r = np.sqrt((x[0] - 3)**2 + (x[0] - 2)**2)
    return np.array(((x[0] - 3)/r, (x[0] - 2)/r))

optimize.minimize(f, np.array([0, 0]), method="L-BFGS-B",
                     bounds=((-1.5, 1.5), (-1.5, 1.5)))

accumulated = np.array(accumulator)
plt.plot(accumulated[:, 0], accumulated[:, 1])

In [ ]:
# accumulated has the search paths
accumulated

### Example 2

Consider the function

$$ f(x_0, x_1) = \sqrt{ (x_0-3)^2 + (x_1 - 2)^2 } $$

subject to the constrains: $g(x_0, x_1) \geq 0$ where

$$ g(x_0, x_1) = 1.5 - ( \left| x_0 \right| + \left| x_1 \right|)$$.

$g \geq 0$ is the shaded region plotted below. (Quiz: why?)

In [ ]:
# Plot the geometry of g

plt.figure()
plt.plot([-1.5,    0,  1.5,    0, -1.5],
        [   0,  1.5,    0, -1.5,    0], 'k', linewidth=2)
plt.fill_between([ -1.5,    0,  1.5],
                [    0, -1.5,    0],
                [    0,  1.5,    0],
                color='.8')
plt.axvline(0, color='k', linewidth=1)
plt.axhline(0, color='k', linewidth=1)
plt.text(0, 1.7, '$x_2$', size=16)
plt.text(1.7,0, '$x_1$', size=16)
plt.axis('off')

`fill_between()`: https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.fill_between.html

#### Preparation

Some of the tricks we will use in the code further below:

In [ ]:
x, y = np.mgrid[-2.03:4.2:.04, -1.6:3.2:.04]
x = x.T
y = y.T
x

In [ ]:
# This function ensures that the result is an numpy array
np.atleast_1d(1.5 - np.sum(np.abs(x)))

In [ ]:
# otherwise
1.5 - np.sum(np.abs(x))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

x, y = np.mgrid[-2.03:4.2:.04, -1.6:3.2:.04]
x = x.T
y = y.T

plt.figure(1, figsize=(8, 5))
plt.clf()
plt.axes([0, 0, 1, 1])

contours = plt.contour(np.sqrt((x - 3)**2 + (y - 2)**2),
                    extent=[-2.03, 4.2, -1.6, 3.2],
                    cmap=plt.cm.gnuplot)
plt.clabel(contours,
        inline=1,
        fmt='%1.1f',
        fontsize=14)
plt.plot([-1.5,    0,  1.5,    0, -1.5],
        [   0,  1.5,    0, -1.5,    0], 'k', linewidth=2)
plt.fill_between([ -1.5,    0,  1.5],
                [    0, -1.5,    0],
                [    0,  1.5,    0],
                color='.8')
plt.axvline(0, color='k')
plt.axhline(0, color='k')

plt.text(-.9, 2.8, '$x_2$', size=20)
plt.text(3.6, -.6, '$x_1$', size=20)
plt.axis('tight')
plt.axis('off')

# And now plot the optimization path
accumulator = list()

def f(x):
    # Store the list of function calls
    accumulator.append(x)
    return np.sqrt((x[0] - 3)**2 + (x[1] - 2)**2)


def constraint(x):
    return np.atleast_1d(1.5 - np.sum(np.abs(x)))

optimize.minimize(f, np.array([0, 0]), method="SLSQP",
                     constraints={"fun": constraint, "type": "ineq"})

accumulated = np.array(accumulator)
plt.plot(accumulated[:, 0], accumulated[:, 1])

In [ ]:
accumulated